# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![table](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1577491200000&hmac=M9BoPKD71JVSyP079GhXaS3ClccJFaqYC3xUFUUYf28)


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [38]:
postal_codes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(postal_codes_url)
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [44]:
df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [64]:
df_boroughs = df[df['Borough'] != 'Not assigned']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [68]:
df_agg = (df_boroughs.groupby('PostalCode')
          .agg({'Borough': 'first', 'Neighborhood': ', '.join})
          .reset_index()
          .reindex(columns=df.columns))
df_agg

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [75]:
df_agg['Neighborhood'] = df_agg.apply(lambda x: x['Borough'] if x['Neighborhood'] == 'Not assigned' else x['Neighborhood'], axis=1)

In [82]:
print(df_agg.to_string())

    PostalCode           Borough                                       Neighborhood
0          M1B       Scarborough                                     Rouge, Malvern
1          M1C       Scarborough             Highland Creek, Rouge Hill, Port Union
2          M1E       Scarborough                  Guildwood, Morningside, West Hill
3          M1G       Scarborough                                             Woburn
4          M1H       Scarborough                                          Cedarbrae
5          M1J       Scarborough                                Scarborough Village
6          M1K       Scarborough        East Birchmount Park, Ionview, Kennedy Park
7          M1L       Scarborough                    Clairlea, Golden Mile, Oakridge
8          M1M       Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9          M1N       Scarborough                        Birch Cliff, Cliffside West
10         M1P       Scarborough  Dorset Park, Scarborough Town Centre, Wexf

In [83]:
df_agg.shape

(103, 3)